# DL2 - CNN


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/cnn">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/cnn.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/cnn.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

On utilise ici un CNN (https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) (CNN) pour claiifier les images [CIFAR ](https://www.cs.toronto.edu/~kriz/cifar.html), avec l'API  [Keras Sequential API](https://www.tensorflow.org/guide/keras/overview), en juste quelques lignes de code.


### Importer TensorFlow

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### Préparer la base CIFAR10

Cette base contient 60 000 images couleur réparties équitablement en 10 classes.
On divise en 50 000 images d'apprentissage, et 10 000 de test, et on les normalise :

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Explorons les données

On fait afficher les 25 premières images avec leurs labels, et les 10 classes qu'on apprend :


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

Et une seule image :

In [ ]:
plt.imshow(train_images[12])
label = train_labels[12]
print(label)


**Création du CNN : **

In [ ]:
print(train_images[11])

On emplile des couches de type [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) et [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layers.

Les images en entrée sont des tenseurs de 32 * 32 * couleur.
Couleur est le code RGB de la couleur du pixel.
D'ou un parametre input_shape qui est de 32 * 32 * 3

Les couches Conv2D sont de la forme :
layers.conv2D(nombre de filtres - ou cartes - , ( largeur du noyau, hauteur du noyau - ou kernel suze).

Les couches Maxpool2D prennent pour parametres la taille de la grille considérée pour sortir le maximum.



In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


Voici l'architecture du réseau :

In [ ]:
model.summary()

La premiere couche mène à 32 cartes 30*30. Ensuite on applique une opération MaxPool sur chacune de ces cartes, avec une grille 2*2 : il en résulte 32 cartes 15*15 : on divise par 4 la taille de l'information.
PLus on va "profond" dans le réseau, plus l'info se résume. Et donc, plus on peut rajouter de couches sans perdre de temps de calcul, car c'est les dimensions des données qui sont couteuses en temps.

### La touche finale : les couches denses "fully connected" :
On rajoute en sortie de la couche conv2D-2 une couche qui permet de faire la classification, elle est totalement connectée.
Il faut dans un premier temps applatir la sortie de la couche conv2D-2 qui est un tenseur en 3 dimensions (4*4*64), pour qu'il devienne un vecteur 1D. On ajoute ensuite une ou plusieurs couches Dense (ici 2).

La base CIFAR a 10 classes à apprendre, il nous faut donc 10 neurones de sortie (softmax).


In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

Voici l'architecture finale du modèle de DL :

In [ ]:
model.summary()

### Compiler et entrainer le modèle

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=100,
                    validation_data=(test_images, test_labels))

### Evaluer le modèle
En bleu, précision sur le set d'apprentissage, en orange sur le set de validation

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

Précision de 70% environ !



On teste maintenant le modèle, avec une image qu'on importe nous même :


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
from tensorflow.keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(32, 32))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)


  print(classes)

classes2 = np.argmax(classes, axis=1)

if classes2[0] == 0:
        print('It is an airplane')
elif classes2[0] == 1:
        print('It is an automobile')
elif classes2[0] == 2:
        print('It is a bird')
elif classes2[0] == 3:
        print('It is a cat')
elif classes2[0] == 4:
        print('It is a deer')
elif classes2[0] == 5:
        print('It is a dog')
elif classes2[0] == 6:
        print('It is a frog')
elif classes2[0] == 7:
        print('It is a horse')
elif classes2[0] == 8:
        print('It is a ship')
elif classes2[0] == 9:
        print('It is a truck')
else:
        print('Can\'t recognize the image')

print('done')